# ⚖️ ВИЗУАЛИЗАЦИЯ БАЛАНСИРОВКИ КЛАССОВ

Визуализация балансировки категорий в датасете


In [ ]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")
plt.rcParams['figure.figsize'] = (15, 10)

viz_dir = 'png/balancing'
os.makedirs(viz_dir, exist_ok=True)

%matplotlib inline


In [ ]:
df_balanced = pd.read_csv('dataset_balanced.csv')
df = df_balanced.copy()

try:
    df_original = pd.read_csv('dataset.csv')
    has_original = True
except:
    has_original = False
    df_original = None


In [ ]:
# Визуализация балансировки (код из metrics_analysis.ipynb, ячейка 7)
fig, axes = plt.subplots(2, 2, figsize=(18, 14))
fig.suptitle('⚖️ ВИЗУАЛИЗАЦИЯ БАЛАНСИРОВКИ КЛАССОВ', fontsize=16, fontweight='bold')

category_counts = df['category'].value_counts().sort_index()
colors = plt.cm.Set3(np.linspace(0, 1, len(category_counts)))

# 1. Распределение категорий
ax1 = axes[0, 0]
bars = ax1.bar(range(len(category_counts)), category_counts.values, color=colors, alpha=0.8, edgecolor='black', linewidth=1.5)
ax1.set_xticks(range(len(category_counts)))
ax1.set_xticklabels(category_counts.index, rotation=45, ha='right', fontsize=9)
ax1.set_ylabel('Количество записей', fontsize=12)
ax1.set_title('Распределение категорий (после балансировки)', fontsize=13, fontweight='bold')
ax1.grid(axis='y', alpha=0.3)
for bar, count in zip(bars, category_counts.values):
    height = bar.get_height()
    ax1.text(bar.get_x() + bar.get_width()/2., height, f'{count:,}\n({count/len(df)*100:.1f}%)',
            ha='center', va='bottom', fontsize=9, fontweight='bold')

# 2. Круговая диаграмма
ax2 = axes[0, 1]
ax2.pie(category_counts.values, labels=category_counts.index, autopct='%1.1f%%', startangle=90, colors=colors, textprops={'fontsize': 9})
ax2.set_title('Процентное распределение категорий', fontsize=13, fontweight='bold')

# 3. Сравнение до/после
if has_original:
    ax3 = axes[1, 0]
    original_counts = df_original['category'].value_counts().sort_index()
    balanced_counts = df['category'].value_counts().sort_index()
    all_categories = sorted(set(original_counts.index) | set(balanced_counts.index))
    original_values = [original_counts.get(cat, 0) for cat in all_categories]
    balanced_values = [balanced_counts.get(cat, 0) for cat in all_categories]
    x = np.arange(len(all_categories))
    width = 0.35
    bars1 = ax3.bar(x - width/2, original_values, width, label='До балансировки', color='#ff6b6b', alpha=0.8, edgecolor='black')
    bars2 = ax3.bar(x + width/2, balanced_values, width, label='После балансировки', color='#51cf66', alpha=0.8, edgecolor='black')
    ax3.set_ylabel('Количество записей', fontsize=12)
    ax3.set_title('Сравнение до и после балансировки', fontsize=13, fontweight='bold')
    ax3.set_xticks(x)
    ax3.set_xticklabels(all_categories, rotation=45, ha='right', fontsize=9)
    ax3.legend(fontsize=10)
    ax3.grid(axis='y', alpha=0.3)
else:
    axes[1, 0].text(0.5, 0.5, 'Исходный датасет\nне найден', ha='center', va='center', fontsize=14)

# 4. Статистика балансировки
ax4 = axes[1, 1]
max_count = category_counts.max()
min_count = category_counts.min()
imbalance_ratio = max_count / min_count if min_count > 0 else 0
balance_data = {'Максимум': max_count, 'Минимум': min_count, 'Среднее': category_counts.mean(), 'Медиана': category_counts.median()}
bars = ax4.bar(balance_data.keys(), balance_data.values(), color=['#ff6b6b', '#51cf66', '#4dabf7', '#ffd43b'], alpha=0.8, edgecolor='black', linewidth=2)
ax4.set_ylabel('Количество записей', fontsize=12)
ax4.set_title(f'Статистика балансировки\n(Коэффициент: {imbalance_ratio:.2f}:1)', fontsize=13, fontweight='bold')
ax4.grid(axis='y', alpha=0.3)
for bar, (key, value) in zip(bars, balance_data.items()):
    height = bar.get_height()
    ax4.text(bar.get_x() + bar.get_width()/2., height, f'{int(value):,}', ha='center', va='bottom', fontsize=11, fontweight='bold')

plt.tight_layout()
save_path = os.path.join(viz_dir, 'balancing_overview.png')
plt.savefig(save_path, dpi=150, bbox_inches='tight')
print(f"✅ График сохранен: {save_path}")
plt.show()
